## **4. Topic modeling** (sklearn)
NNMF    - Non-Negative Matrix Factorization  
LSA     - Latent Semantic Analysis  
LDA     - Latent Dirichlet Allocation

### **Lire les données / termes candidats extraits au prétraitement**

In [57]:
path = '../04-filtrage/output/'
acteur = 'msss'
tag = 'dependances'

if tag:
    csv_file = acteur + '_' + tag + '_significant-collocations.csv'

else:
    csv_file = acteur + '_significant-collocations.csv'

In [58]:
from pandas import *

with open(path+csv_file, encoding='utf-8') as f:
    csv = read_csv(f)[["Collocation", "Structure syntaxique", "Fréquence"]] # "LLR", "p-value"]]

csv

,Collocation,Structure syntaxique,Fréquence
0,répertoire des ressources en dépendances,NOM PRP:det NOM PRP NOM,140
1,fiche de la ressource,NOM PRP DET:ART NOM,118
2,services sociaux,NOM ADJ,88
3,durée du programme,NOM PRP:det NOM,85
4,approche s préconisée,NOM VER:simp VER:pper,83
...,...,...,...
364,réadaptation dans un délai,NOM PRP DET:ART NOM,6
365,indicateurs associés aux egi,NOM VER:pper PRP:det NOM,6
366,accès à un service,NOM PRP DET:ART NOM,6
367,sherbrooke québec,NOM ADJ,6


In [59]:
vocabulaire = [t.lower() for t in list(csv['Collocation'])]

In [60]:
import os, shutil, re
from pathlib import Path
from os import path
from pandas import *

base_path = '../03-corpus/2-data/1-fr/'
if tag:
    base_path = path.join(base_path, acteur, acteur + '_' + tag + '.csv')

else:
    base_path = path.join(base_path, acteur +  '.csv')
        
with open(base_path, "r", encoding = "UTF-8") as f:
    data = read_csv(base_path, sep=',')
    text = data['text'].tolist()

nb_docs = len(text)

In [61]:
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    
punct = '[!#$%&\(\)•►*+,-\/:;<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

corpus = [re.sub(punct, ' ', t).replace("' ", "'" ).replace("'", "'") for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in corpus]
corpus = [re.sub(spaces, ' ', t) for t in corpus]
corpus = [re.sub(phones, ' ', t) for t in corpus]
corpus = [re.sub(postals, ' ', t) for t in corpus]
corpus = [re.sub(punct, ' ', t) for t in corpus]
corpus = [t.replace("  ", " " ) for t in corpus]

### **Vectorisation / pondération TF-IDF**

In [76]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import numpy as np

# NMF and LSA are able to use tf-idf
tfidf_vectorizer = TfidfVectorizer(vocabulary = set(vocabulaire),ngram_range=(1,12), use_idf=True, lowercase=False)
tfidf = tfidf_vectorizer.fit_transform(corpus)
tfidf_features_names = tfidf_vectorizer.get_feature_names_out()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(vocabulary = set(vocabulaire), ngram_range=(1,12))
tf = tf_vectorizer.fit_transform(corpus)
tf_features_names = tf_vectorizer.get_feature_names()

# LSA -  
# input_matrix = tfidf_vectorizer.fit_transform(corpus).todense()

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [88]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
no_topics = 100

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

# Run LSA
lsa = TruncatedSVD(n_components=no_topics, algorithm='randomized', n_iter=100, random_state=122).fit(tfidf)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/decomposition/_nmf.py:1477: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  warnings.warn(


In [89]:
def display_topics(model, feature_names, no_top_words):
    tab = []
    for topic_idx, topic in enumerate(model.components_):
        #print("Topic %d:" % (topic_idx))
        tab.append(" - ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]))
    
    #tab = [" ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]) for topic in enumerate(model.components_)]
    df = DataFrame(tab, columns=['Topic']) # Éventuellement on voudrait remplacer l'index par un label plus significatif
    
    base_path = '../06-clustering/'
    if tag:
        output_file = acteur + '_' + tag + '_clusters_' + (str(model).split('(')[0]) +'.csv'
    else:
        output_file = acteur + '_clusters_' + (str(model).split('(')[0]) +'.csv'
    df.to_csv(base_path + output_file)
    return df


no_top_words = 30

In [90]:
print("Topic Modeling avec LSA")
display_topics(lsa, tfidf_features_names, no_top_words)

Topic Modeling avec LSA


,Topic
0,durée du programme - ressource certifiée en dé...
1,services de réadaptation en externe - services...
2,jeu pathologique - toxicomanie pour les problè...
3,capitale nationale - capitale nationale québec...
4,centre du québec - mauricie et centre du québe...
...,...
95,aide et soutien à la récupération - accès aux ...
96,argent chez les jeunes - aide et soutien à l'e...
97,attente d'un service en clsc - appel à projets...
98,accès aux services de santé - attestation temp...


In [91]:
print("Topic Modeling avec NMF")
display_topics(nmf, tfidf_features_names, no_top_words)

Topic Modeling avec NMF


,Topic
0,thérapie individuelle - thérapie individuelle ...
1,service de soutien - désintoxication en extern...
2,problèmes de jeu - problèmes de jeu pathologiq...
3,capitale nationale - capitale nationale québec...
4,centre du québec - mauricie et centre - mauric...
...,...
95,évaluation des conducteurs saaq dernière - gro...
96,évaluation des conducteurs saaq dernière - gro...
97,évaluation des conducteurs saaq dernière - gro...
98,évaluation des conducteurs saaq dernière - gro...


In [92]:
print("Topic Modeling avec LDA")
display_topics(lda, tf_features_names, no_top_words)

Topic Modeling avec LDA


,Topic
0,réinsertion sociale - répertoire des ressource...
1,répertoire des ressources en dépendances - dur...
2,estrie fiche - côte nord - innovation en matiè...
3,santé mentale - clientèle jeunes - fédération ...
4,réadaptation en dépendance - service de soutie...
...,...
95,situation d'itinérance - engagement social - s...
96,côte nord - jeunes à faire - ressource à but l...
97,centre du québec - mauricie et centre du québe...
98,services sociaux - ministère de la santé et de...
